In [1]:
import pandas as pd
import numpy as np
from os.path import join
from pathlib import Path
import os


In [2]:
path = Path("data/")
filename_1 = 'train_dataset.csv'
filename_2 = 'preprocessing_loaded_table.csv'
file_source = 'data/VVP.csv'




In [3]:
#Функция для возврата часа в дату
def date_plus_hours_to_index(data):
    data = data.copy()
    data['date'] = data['date'] + pd.to_timedelta(data['time'] , 'H')
    data = data.set_index('date')
    return data

In [4]:
# Функция для сдвига на суткм (в скаченном датасете разбивка по 30 мин, по этому timeshift=48)
def shift_features_fact(df, timeshift=48):
    list_fact_columns=list([df.columns])
    new_df = df.copy()
    for column in list_fact_columns:
        new_df[column] = new_df[column].shift(timeshift)
    
    return new_df

In [5]:
# объединение оригинальных тренировочных данных и скаченных признаков со сдвигом и ввп
def data_merge(path, filename_1, filename_2, file_source):
    df_train = pd.read_csv(join(path, filename_1), parse_dates=[0])
    df_train = date_plus_hours_to_index(df_train)
    df_fich = pd.read_csv(join(path, filename_2))
    df_fich['date'] = pd.to_datetime(df_fich['date'], infer_datetime_format=True)
    df_fich = df_fich.set_index('date')
    df_fich_shift = shift_features_fact(df_fich)
    df_train = pd.merge(df_train, df_fich_shift[df_fich.columns], left_index=True, right_index=True)

    vvp = pd.read_csv(file_source)
    vvp['date'] = pd.to_datetime(vvp['date'], format ='%Y-%m-%d %H:%M:%S')
    vvp.drop('for_month',axis=1,inplace=True) 
    vvp.set_index('date',inplace=True)

    df_train = df_train.join(vvp,how='left')
    df_train['VVP_perc'].fillna(method = 'ffill',inplace=True)
    df_train['VVP_perc'].fillna(value = vvp['VVP_perc'].iloc[-1],inplace=True)

    



    return df_train

    
    

In [6]:
df_train_shift = data_merge(path, 'train_dataset.csv', 'preprocessing_loaded_table.csv', 'data/VVP.csv')


In [7]:
df_train_shift.to_csv('data/df_train_vvp_shift.csv')

In [8]:
df_train_shift

,time,target,temp,temp_pred,weather_pred,weather_fact,P,U,WW,Td,N,S,W,E,VVP_perc
date,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0,481.510,2.9,2.0,"пасм, ветер",ветер,763.5,100.0,слабый дождь,2.0,1.0,0.0,0.0,0.0,1.8
2019-01-01 01:00:00,1,462.872,2.9,2.0,"пасм, ветер",ветер,764.3,93.0,слабый дождь,1.0,1.0,0.0,0.0,0.0,1.8
2019-01-01 02:00:00,2,449.718,2.9,2.0,"пасм, ветер",ветер,765.0,93.0,нет осадков,2.0,1.0,0.0,0.0,0.0,1.8
2019-01-01 03:00:00,3,430.908,4.3,2.0,"пасм, ветер","ветер, пасм",765.8,87.0,нет осадков,1.0,1.0,0.0,0.0,0.0,1.8
2019-01-01 04:00:00,4,415.163,4.3,2.0,"пасм, ветер","ветер, пасм",766.6,87.0,нет осадков,1.0,1.0,0.0,0.0,0.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31 19:00:00,19,552.960,7.9,6.0,"пасм, 61%","пасм, дымка",753.9,82.0,слабый дождь,6.0,0.0,1.0,0.0,0.5,-3.1
2023-03-31 20:00:00,20,563.985,7.9,6.0,"пасм, 61%","пасм, дымка",753.1,93.0,слабый дождь,7.0,0.0,1.0,0.0,0.5,-3.1
2023-03-31 21:00:00,21,560.191,6.3,5.0,"пасм, 61%","пасм, дымка",753.1,93.0,слабый дождь,7.0,0.0,1.0,0.0,1.0,-3.1
